In [ ]:
import numpy as np
import pandas as pd

from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors
import collections

from scipy.stats import chisquare
from scipy.stats import mannwhitneyu
from scipy.stats import wilcoxon
from scipy.stats import ks_2samp

from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
%matplotlib inline

# Apply $X^{2}$ test

In [ ]:
def fdr_sample_qs(data, batch, sample_index, k = 30):
    
    """
    data= dataframe of gene experession data, rows are samples and columns are genes
    batch = list of batch labels corresponding to the samples in the data
    sample_index = the index of the sample that we want to find FDR of its neighborhood
    k = k nearest neighbors
    """
    
    #conver dataframe to array
    X = np.asarray(data)
    
    # find 50 first PCAs
    
    pca = PCA(n_components=50)
    PCs = pca.fit_transform(X)
    
    # k nearest neighbors of each PC
    
    nbrs = NearestNeighbors(n_neighbors=k, algorithm='ball_tree').fit(PCs)
    distances, indices = nbrs.kneighbors(PCs)
    
    #list of neighbors of sample s
    nbrs_of_s = indices[sample_index] 
    
    # what batches each neighbor of sample s belong to
    batches_nbrs_s = [batch[index] for index in list(nbrs_of_s)]

    # find frequency of each batch in nbrs_s
    counter=collections.Counter(batches_nbrs_s)
    freq_nbrs_s= list(counter.values())

    counter=collections.Counter(batch)
    n_samples = len(batch)
    n_batch_1 = counter[1]
    n_batch_2 = counter[2]

    counter=collections.Counter(batch)
    s, p = chisquare(f_obs = freq_nbrs_s,  f_exp =[k*n_batch_1/n_samples, k*n_batch_2/n_samples])

    return p

# Apply Mann-Whitney U test

In [ ]:
def fdr_sample_mw(data, batch, sample_index, k):
    
    """
    data= dataframe of gene experession data, rows are samples and columns are genes
    batch = list of batch labels corresponding to the samples in the data
    sample_index = the index of the sample that we want to find FDR of its neighborhood
    k = k nearest neighbors
    """
        
    # find 50 first PCAs
    pca = PCA(n_components=50)
    PCs = pca.fit_transform(data)
    
    # k nearest neighbors of each PC
    nbrs = NearestNeighbors(n_neighbors=k, algorithm='ball_tree').fit(PCs)
    distances, indices = nbrs.kneighbors(PCs)
    
    #list of neighbors of sample s
    nbrs_of_s = indices[sample_index] 
    
    # what batches each neighbor of sample s belong to
    batches_nbrs_s = [batch[index] for index in list(nbrs_of_s)]

    counter=collections.Counter(batch)
    n_samples = len(batch)
    n_batch_1 = counter[1]
    n_batch_2 = counter[2]
    
    # frequency of the overall dataset
    freq = [n_batch_1/n_samples, n_batch_2/n_samples]
    
    # frequency of the overall dataset with the length k
    expected_distr = [1 for i in range(round(k * freq[0]))] + [2 for i in range(round(k * freq[1]))]

    s, p = mannwhitneyu(batches_nbrs_s,  expected_distr)

    return p

# Apply Wilcoxon Signed-Rank test

In [ ]:
def fdr_sample_w(data, batch, sample_index, k):
    
    """
    data= dataframe of gene experession data, rows are samples and columns are genes
    batch = list of batch labels corresponding to the samples in the data
    sample_index = the index of the sample that we want to find FDR of its neighborhood
    k = k nearest neighbors
    """
        
    # find 50 first PCAs
    pca = PCA(n_components=50)
    PCs = pca.fit_transform(data)
    
    # k nearest neighbors of each PC
    nbrs = NearestNeighbors(n_neighbors=k, algorithm='ball_tree').fit(PCs)
    distances, indices = nbrs.kneighbors(PCs)
    
    #list of neighbors of sample s
    nbrs_of_s = indices[sample_index] 
    
    # what batches each neighbor of sample s belong to
    batches_nbrs_s = [batch[index] for index in list(nbrs_of_s)]

    counter=collections.Counter(batch)
    n_samples = len(batch)
    n_batch_1 = counter[1]
    n_batch_2 = counter[2]
    
    # frequency of the overall dataset
    freq = [n_batch_1/n_samples, n_batch_2/n_samples]
    
    # frequency of the overall dataset with the length k
    expected_distr = [1 for i in range(round(k * freq[0]))] + [2 for i in range(round(k * freq[1]))]

    s, p = wilcoxon(batches_nbrs_s,  expected_distr)

    return p

# Kolmogorov-Smirnov's test

In [ ]:
def fdr_sample_ks(data, batch, sample_index, k = 20):
    
    """
    data= data as array of gene experession data, rows are samples and columns are genes
    batch = list of batch labels corresponding to the samples in the data
    sample_index = the index of the sample that we want to find FDR of its neighborhood
    k = k nearest neighbors
    """
        
    # find 50 first PCAs
    pca = PCA(n_components=50)
    PCs = pca.fit_transform(data)
    
    # k nearest neighbors of each PC
    nbrs = NearestNeighbors(n_neighbors=k, algorithm='ball_tree').fit(PCs)
    distances, indices = nbrs.kneighbors(PCs)
    
    #list of neighbors of sample s
    nbrs_of_s = indices[sample_index] 
        
    # what batches each neighbor of sample s belong to
    batches_nbrs_s = [batch[index] for index in list(nbrs_of_s)]

    counter=collections.Counter(batch)
    n_samples = len(batch)
    n_batch_1 = counter[1]
    n_batch_2 = counter[2]
    
    # frequency of the overall dataset
    freq = [n_batch_1/n_samples, n_batch_2/n_samples]
    
    # frequency of the overall dataset with the length k
    expected_distr = [1 for i in range(round(k * freq[0]))] + [2 for i in range(round(k * freq[1]))]

    s, p = ks_2samp(batches_nbrs_s,  expected_distr)

    return p

# Simulations

In [ ]:
def simulation(dir_weak, dir_mild, dir_strong, n_batch_1, n_batch_2, alpha = 0.05, k = 20):
  data_weak = pd.read_csv(dir_weak) 
  data_weak = data_weak.rename(columns={ data_weak.columns[0]: "Genes" })
  data_weak = data_weak.set_index("Genes")
  data_weak = data_weak.transpose()

  data_mild = pd.read_csv(dir_mild) 
  data_mild = data_mild.rename(columns={ data_mild.columns[0]: "Genes" })
  data_mild = data_mild.set_index("Genes")
  data_mild = data_mild.transpose()  

  data_strong = pd.read_csv(dir_strong) 
  data_strong = data_strong.rename(columns={data_strong.columns[0]: "Genes" })
  data_strong = data_strong.set_index("Genes")
  data_strong = data_strong.transpose()

  batch_1 = [1 for i in range(n_batch1)]
  batch_2 = [2 for i in range(n_batch2)]

  #Assign the first n_batch1 samples to batch 1 and the next n_batch2 samples to batch 2
  batch = batch_1 + batch_2
  n_samples = len(batch)

  ##### Chi-square test #####

  # false discovery rate
  FDR_CHI_SQU ={}

  # rejection rate
  RR_CHI = {}

  # list of p-value
  p_CHI_weak = [fdr_sample_qs(data_weak, batch, i, k) for i in range(n_samples)]
  p_CHI_mild = [fdr_sample_qs(data_mild, batch, i, k) for i in range(n_samples)]
  p_CHI_strong = [fdr_sample_qs(data_strong, batch, i, k) for i in range(n_samples)]

  # fdr
  FDR_CHI_SQU['weak'] = 1-np.mean(p_CHI_weak)
  FDR_CHI_SQU['mild'] = 1-np.mean(p_CHI_mild)
  FDR_CHI_SQU['strong'] = 1-np.mean(p_CHI_strong)
  print("FDR_CHI_SQU: ", FDR_CHI_SQU)

  # rejection rate
  RR_CHI_SQU['weak'] = sum(1 for i in p_CHI_weak if i < alpha)/len(p_CHI_weak)
  RR_CHI_SQU['mild'] = sum(1 for i in p_CHI_mild if i < alpha)/len(p_CHI_mild)
  RR_CHI_SQU['strong'] = sum(1 for i in p_CHI_strong if i < alpha)/len(p_CHI_strong)
  print("RR_CHI_SQU: ", RR_CHI_SQU)


  ##### Mann-Whitney U test #####
  FDR_M_W = {}  
  RR_M_W = {} 

  # list of p-value
  p_M_W_weak = [fdr_sample_mw(data_weak, batch, i, k) for i in range(n_samples)]
  p_M_W_mild = [fdr_sample_mw(data_mild, batch, i, k) for i in range(n_samples)]
  p_M_W_strong = [fdr_sample_mw(data_strong, batch, i, k) for i in range(n_samples)]

  # fdr
  FDR_M_W['weak'] = 1-np.mean(p_M_W_weak)
  FDR_M_W['mild'] = 1-np.mean(p_M_W_mild)
  FDR_M_W['strong'] = 1-np.mean(p_M_W_strong)
  print("FDR_M_W: ", FDR_M_W)

  # rejection rate
  RR_M_W['weak'] = sum(1 for i in p_M_W_weak if i < alpha)/len(p_M_W_weak)
  RR_M_W['mild'] = sum(1 for i in p_M_W_mild if i < alpha)/len(p_M_W_mild)
  RR_M_W['strong'] = sum(1 for i in p_M_W_strong if i < alpha)/len(p_M_W_strong)
  print("RR_M_W: ", RR_M_W)


  ##### Wilcoxon Signed-Rank test #####
  FDR_W = {} 
  RR_W = {}  

  # list of p-value
  p_W_weak = [fdr_sample_w(data_weak, batch, i, k) for i in range(n_samples)]
  p_W_mild = [fdr_sample_w(data_mild, batch, i, k) for i in range(n_samples)]
  p_W_strong = [fdr_sample_w(data_strong, batch, i, k) for i in range(n_samples)]

  # fdr
  FDR_W['weak'] = 1-np.mean(p_W_weak)
  FDR_W['mild'] = 1-np.mean(p_W_mild)
  FDR_W['strong'] = 1-np.mean(p_W_strong)
  print("FDR_W: ", FDR_W)

  # rejection rate
  RR_W['weak'] = sum(1 for i in p_W_weak if i < alpha)/len(p_W_weak)
  RR_W['mild'] = sum(1 for i in p_W_mild if i < alpha)/len(p_W_mild)
  RR_W['strong'] = sum(1 for i in p_W_strong if i < alpha)/len(p_W_strong)
  print("RR_W: ", RR_W)

 
  ##### Kolmogorov-Smirnov's test #####
  FDR_K_S = {} 
  RR_K_S = {}  

  # list of p-value
  p_K_S_weak = [fdr_sample_ks(data_weak, batch, i, k) for i in range(n_samples)]
  p_K_S_mild = [fdr_sample_ks(data_mild, batch, i, k) for i in range(n_samples)]
  p_K_S_strong = [fdr_sample_ks(data_strong, batch, i, k) for i in range(n_samples)]

  # fdr
  FDR_K_S['weak'] = 1-np.mean(p_K_S_weak)
  FDR_K_S['mild'] = 1-np.mean(p_K_S_mild)
  FDR_K_S['strong'] = 1-np.mean(p_K_S_strong)
  print("FDR_K_S: ", FDR_K_S)

  # rejection rate
  RR_K_S['weak'] = sum(1 for i in p_K_S_weak if i < alpha)/len(p_K_S_weak)
  RR_K_S['mild'] = sum(1 for i in p_K_S_mild if i < alpha)/len(p_K_S_mild)
  RR_K_S['strong'] = sum(1 for i in p_K_S_strong if i < alpha)/len(p_K_S_strong)
  print("RR_K_S: ", RR_K_S)

  return   FDR_CHI_SQU, RR_CHI,  FDR_M_W, RR_M_W, FDR_W, RR_W, FDR_K_S, RR_K_S

In [ ]:
def merge_dict(FDR_CHI_SQU, FDR_M_W, FDR_W, FDR_K_S, RR_CHI_SQU, RR_M_W, RR_W, RR_K_S):
  ds_fdr = [FDR_CHI_SQU, FDR_M_W, FDR_W, FDR_K_S]
  ds_rr = [RR_CHI_SQU, RR_M_W, RR_W, RR_K_S]
  d_fdr = {}
  ds_rr = {}

  for k in FDR_CHI_SQU.keys():
    d_fdr[k] = tuple(d_fdr[k] for d in ds_fdr)
    d_rr[k] = tuple(d_rr[k] for d in ds_rr)

  # convert dictionary to panda dataframe
  FDR_result = pd.DataFrame.from_dict(d_fdr, orient='index',
                       columns=['FDR_CHI', 'FDR_M_W', 'FDR_W', 'FDR_K_S'])
  RR_result = pd.DataFrame.from_dict(d_rr, orient='index',
                       columns=['RR_CHI', 'RR_M_W', 'RR_W', 'RR_K_S'])
  return FDR_result, RR_result


# Simulation 1:1 ratio

In [ ]:
n_batch_1 = 250
n_batch_2 = 250

dir_weak = 'weak/df.sim.weak.1.1.csv'
dir_mild = 'mild/df.sim.mild.1.1.csv'
dir_strong = 'strong/df.sim.strong.1.1.csv'
FDR_CHI_SQU, RR_CHI,  FDR_M_W, RR_M_W, FDR_W, RR_W, FDR_K_S, RR_K_S = simulation(dir_weak, dir_mild, dir_strong, n_batch_1, n_batch_2, alpha = 0.05, k =20)
FDR_result, RR_result = merge_dict(FDR_CHI_SQU, FDR_M_W, FDR_W, FDR_K_S, RR_CHI_SQU, RR_M_W, RR_W, RR_K_S)
FDR_result.to_csv('FDR_result_ratio_1_1.csv', index=True)
RR_result.to_csv('RR_result_ratio_1_1.csv', index=True)

# Simulation 1:3 ratio

In [ ]:
n_batch_1 = 500/4
n_batch_2 = 500*(3/4)

dir_weak = 'weak/df.sim.weak.1.3.csv'
dir_mild = 'mild/df.sim.mild.1.3.csv'
dir_strong = 'strong/df.sim.strong.1.3.csv'
FDR_CHI_SQU, RR_CHI,  FDR_M_W, RR_M_W, FDR_W, RR_W, FDR_K_S, RR_K_S = simulation(dir_weak, dir_mild, dir_strong, n_batch_1, n_batch_2, alpha = 0.05, k =20)
FDR_result, RR_result = merge_dict(FDR_CHI_SQU, FDR_M_W, FDR_W, FDR_K_S, RR_CHI_SQU, RR_M_W, RR_W, RR_K_S)
FDR_result.to_csv('FDR_result_ratio_1_3.csv', index=True)
RR_result.to_csv('RR_result_ratio_1_3.csv', index=True)

# Simulation 1:4 ratio

In [ ]:
n_batch_1 = 500/5
n_batch_2 = 500*(4/5)

dir_weak = 'weak/df.sim.weak.1.4.csv'
dir_mild = 'mild/df.sim.mild.1.4.csv'
dir_strong = 'strong/df.sim.strong.1.4.csv'
FDR_CHI_SQU, RR_CHI,  FDR_M_W, RR_M_W, FDR_W, RR_W, FDR_K_S, RR_K_S = simulation(dir_weak, dir_mild, dir_strong, n_batch_1, n_batch_2, alpha = 0.05, k =20)
FDR_result, RR_result = merge_dict(FDR_CHI_SQU, FDR_M_W, FDR_W, FDR_K_S, RR_CHI_SQU, RR_M_W, RR_W, RR_K_S)
FDR_result.to_csv('FDR_result_ratio_1_4.csv', index=True)
RR_result.to_csv('RR_result_ratio_1_4.csv', index=True)

# Simulation 1:9 ratio

In [ ]:
n_batch_1 = 500/10
n_batch_2 = 500*(9/10)

dir_weak = 'weak/df.sim.weak.1.9.csv'
dir_mild = 'mild/df.sim.mild.1.9.csv'
dir_strong = 'strong/df.sim.strong.1.9.csv'
FDR_CHI_SQU, RR_CHI,  FDR_M_W, RR_M_W, FDR_W, RR_W, FDR_K_S, RR_K_S = simulation(dir_weak, dir_mild, dir_strong, n_batch_1, n_batch_2, alpha = 0.05, k =20)
FDR_result, RR_result = merge_dict(FDR_CHI_SQU, FDR_M_W, FDR_W, FDR_K_S, RR_CHI_SQU, RR_M_W, RR_W, RR_K_S)
FDR_result.to_csv('FDR_result_ratio_1_9.csv', index=True)
RR_result.to_csv('RR_result_ratio_1_9.csv', index=True)

# Simulation 1:19 ratio

In [ ]:
n_batch_1 = 500/20
n_batch_2 = 500*(19/20)

dir_weak = 'weak/df.sim.weak.1.19.csv'
dir_mild = 'mild/df.sim.mild.1.19.csv'
dir_strong = 'strong/df.sim.strong.1.19.csv'
FDR_CHI_SQU, RR_CHI,  FDR_M_W, RR_M_W, FDR_W, RR_W, FDR_K_S, RR_K_S = simulation(dir_weak, dir_mild, dir_strong, n_batch_1, n_batch_2, alpha = 0.05, k =20)
FDR_result, RR_result = merge_dict(FDR_CHI_SQU, FDR_M_W, FDR_W, FDR_K_S, RR_CHI_SQU, RR_M_W, RR_W, RR_K_S)
FDR_result.to_csv('FDR_result_ratio_1_19.csv', index=True)
RR_result.to_csv('RR_result_ratio_1_19.csv', index=True)